# lidar_tools

> Tools to examine bathy lidar data extracted using Global Mapper.

In [1]:
#| default_exp lidar_tools

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
import numpy as   np
import math as    m
import pandas as  pd

from bokeh.layouts  import gridplot
from bokeh.palettes import HighContrast3
from bokeh.plotting import figure, show
from bokeh.io       import output_notebook
from bokeh.models   import Span, VSpan
from bokeh.models   import ColumnDataSource, NumeralTickFormatter, HoverTool

### Constants

In [4]:
#| export
ms2knots   = 1.94384
m2ft       = 3.28084
ft2m       = 1/m2ft
deg2rad    = np.pi/180.0
rad2deg    = 180.0/np.pi

### def swath_width( full_scan_angle_d:float, altitude_ft=None, altitude_m=None):

In [5]:
#| export
def swath_width( 
                full_scan_angle_d:float, # Full scan angle in degrees.
                altitude_ft=None,        # Sensor altitude in feet.
                altitude_m=None          # Sensor altitude in meters.
               ) -> float:               # Returns the swath width in meters.
  """
  Compute the scan width give the full scan angle in degrees, and the altitude in either meters
  or feet.  Returns scan width in meters.
  """
  if altitude_ft is not None:
    altitude_m = altitude_ft * ft2m
  elif altitude_m is not None:
    altitude_ft = altitude_m * m2ft
  else:
    rv = None
  if altitude_m is not None:
    rv = 2 * np.tan( full_scan_angle_d * 0.5 * deg2rad ) * altitude_m
  return rv

#### Test swath_width with altitudes in feet and meters

In [6]:
print(swath_width( 16, altitude_ft =10_500 ), 
      swath_width( 16, altitude_m  =3_200 ),
     )

899.5737459767073 899.4613420953053


### class DATA():

In [7]:
#| export
class DATA():
    """
    This class gets populated 
    """
    def __init__(self):
        pass

### class TEST_AREA():

In [30]:
#| export
class TEST_AREA():
    """
    Presets parameters for analysis. Most of the parameters are presets for Bokeh
    plots used in the analysis.  Additional attributes to define datasets and 
    their related attributes are added after this class is instantiated.
    """
    def __init__(self):
        self.title                            = 'Title Not set'
        self.y_axis_label                     = 'not Set'
        self.xaxis_axis_label                 = "NAVD88 Depth (m)"
        self.bin_size                         = 0.01
        self.width                            = 800
        self.height                           = 400
        self.axis_axis_label_text_font_style  = 'bold'
        self.axis_axis_label_text_font_size   = "18pt"
        self.title_text_font_size             = "24pt"
        self.title_text_font_style            = "bold"
        self.axis_major_label_text_font_size  = "16pt"
        self.axis_major_label_text_font_style = "bold"
        self.area_width                       = 0
        self.area_length                      = 0
        self.area                             = 0
        self.data_list                        = []
        self.ref_mean                         = None
        self.colab                            = 'google.colab' in str(get_ipython())

        # Generate the Histogram values.
        self.density = False
        if self.density:
            self.yaxis_axis_label = "Normilized Hits/bin"
        else:
            self.yaxis_axis_label = "Number of Depths"


In [24]:
help(TEST_AREA)

Help on class TEST_AREA in module __main__:

class TEST_AREA(builtins.object)
 |  Presets parameters for analysis. Most of the parameters are presets for Bokeh
 |  plots used in the analysis.  Additional attributes to define datasets and 
 |  their related attributes are added after this class is instantiated.
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables
 |  
 |  __weakref__
 |      list of weak references to the object



In [25]:
ta = TEST_AREA()

In [26]:
vars( ta )

{'title': 'Title Not set',
 'y_axis_label': 'not Set',
 'xaxis_axis_label': 'NAVD88 Depth (m)',
 'bin_size': 0.01,
 'width': 800,
 'height': 400,
 'axis_axis_label_text_font_style': 'bold',
 'axis_axis_label_text_font_size': '18pt',
 'title_text_font_size': '24pt',
 'title_text_font_style': 'bold',
 'axis_major_label_text_font_size': '16pt',
 'axis_major_label_text_font_style': 'bold',
 'area_width': 0,
 'area_length': 0,
 'area': 0,
 'data_list': [],
 'ref_mean': None,
 'colab': False,
 'density': False,
 'yaxis_axis_label': 'Number of Depths'}

### def data( tdf, cls ):

In [12]:
#| export
def data( 
    tdf,             # The parent `TEST_AREA` class.
    fn     ='',      # Data file name
    name   = '',     # Name of this data set
    ref    = False,  # Use as the reference dataset or not.
    scale  = 1.0,    # Scale factor.
    offset = 0.0,    # Offset to apply.
    color  = "blue",
    width  = 3,      # line width
):
    """
    Reads the ASCII data file `cls.fn`, stores the data in `cls.df`, and generates stats
    on the `cls.['ELEVATION']` data.  It also generates a histogram, `cls.hist`, based on `tdf.bin_size`.
    """
    cls        = DATA( )
    cls.parent = tdf
    cls.fn     = fn
    cls.name   = name
    cls.ref    = ref
    cls.offset = offset
    cls.color  = color
    cls.width  = width
    cls.df     = pd.read_csv( cls.fn )
    cls.scale  = scale
    elev       = cls.df['ELEVATION'] = (cls.df['ELEVATION']  + offset) * scale
    cls.mean   = elev.mean()
    cls.std    = elev.std()
    cls.min    = elev.min()
    cls.max    = elev.max()
    cls.p2p    = cls.max - cls.min
    cls.nbins  = int(cls.p2p / tdf.bin_size) + 1
    cls.bins   = np.linspace( cls.min, cls.max, cls.nbins )
    cls.number_of_points = elev.shape[0]
    cls.hist, junk = np.histogram( elev, density = tdf.density, bins = cls.bins)
    if tdf.area >0:
        cls.ppm = cls.number_of_points / tdf.area
    else:
        cls.ppm = 0.0;
    tdf.data_list.append( cls )
    #return cls

### def gen_plot( test_area ):

In [13]:
#| export
def gen_plot( 
    test_area   # Generate and configure a Bokeh plot.
):
    """
    Generate a plot figure, and store it as an attribute in `test-area`. Configure it with preferred settings.
    """
    # Format the tooltip
    tooltips = [
                ('Depth', '$x')  
               ]
    test_area.p = figure(width=test_area.width, height=test_area.height, title = test_area.title )
    test_area.p.axis.axis_label_text_font_style  = test_area.axis_axis_label_text_font_style
    test_area.p.axis.axis_label_text_font_size   = test_area.axis_axis_label_text_font_size
    test_area.p.title.text_font_size             = test_area.title_text_font_size
    test_area.p.title.text_font_style            = test_area.title_text_font_style
    test_area.p.axis.major_label_text_font_size  = test_area.axis_major_label_text_font_size
    test_area.p.axis.major_label_text_font_style = test_area.axis_major_label_text_font_style
    test_area.p.y_range.start                    = 0
    test_area.p.xaxis.axis_label                 = test_area.xaxis_axis_label
    test_area.p.yaxis.axis_label                 = test_area.yaxis_axis_label
    test_area.p.add_tools(HoverTool(tooltips=tooltips))

### def plot_hists( obj ):

In [14]:
#| export
def plot_hists( 
    obj         # Data object.
):
    """
    Plot histograms of each dataset.
    """
    output_notebook()           # Enable notebook for Bokeh output.
    gen_plot( obj )
    for i in obj.data_list:
        obj.p.line( 
            i.bins[0:-1],  
            i.hist, 
            legend_label=i.name,  
            line_width = i.width, 
            color = i.color )
        g = VSpan( x = i.mean, line_color = i.color , line_alpha=.5)
        obj.p.add_glyph(g)
    show(obj.p)  

### def show_attributes( obj ):

In [15]:
#| export
def show_attributes( obj ):
    """
    Printout atrributes of `obj`.  `obj` is a class `DATA`.
    """
    # Show each data set.
    for i in obj.data_list:
        print(
            f'{i.fn:>35s} {i.name:>16s} {i.offset:6.2f}'
            f'{i.mean:6.2f} {i.max-i.min:6.2f}'
            f'{i.color:>8s} {i.width:3d} '
            f'{i.bins[1]-i.bins[0]:6.4f} '
            f'{i.ref}  '
        )

### def gen_stats( v, name='', header=False, area=0 ):

In [16]:
#| export
def gen_stats( 
    data_obj=None,       # The data obj 
    header:bool=False,   # Print a header showing the column names.
    area=0               # The area (in meters) that the values came from.  Used to computed points per meter.
) ->tuple:               # Tuple containing: mean, std, min, max, number_of_points
    """
    Print the statistics for the given elevation data from data_obj. Setting header=True will cause it to 
    print a header describing the columns before printing the results.
    """
    ppm_header = "Points/m"
    if header:
        print(
            f"                        Ref            Std                                 Total\n"
            f"    Data Source  Ref    Dif  Mean(m)  Dev(m)   Min(m)    Max(m)   P2P(m)  Points  {ppm_header}  Scale  Offset(m)"
        )
    else:
        values = data_obj.df['ELEVATION']
        mean = values.mean()
        std  = values.std()
        min  = values.min()
        max  = values.max()
        p2p  = max - min
        number_of_points = values.shape[0]

        if data_obj.ref:
            ref = "<--"
            data_obj.parent.ref_mean = mean
        else:
            ref = "   "

        if data_obj.parent.ref_mean:
            ref_dif = f'{data_obj.parent.ref_mean - mean:7.3f}'
        else:
            ref_dif = f' xxxxxx '
    
        if area >0:
            show_ppm = True
            ppm_header = "Points/m"
            ppm        = number_of_points / area
            ppm_s      = f"{ppm:8.3f}"
        else:
            ppm_header = ''
            ppm_s      = ''
    
        print(
            f'{data_obj.name:>15s} '
            f'{ref:3s} '
            f'{ref_dif:6s} '
            f'{mean:6.3f}  {std:7.3f} {min:8.3f}  {max:8.3f} '
            f'{p2p:8.3f} {number_of_points:7d}  {ppm_s}  {data_obj.scale:5.4f} '
            f'{data_obj.offset:5.3f}' 
        )
        return mean, std, min, max, number_of_points

### def gen_all_stats( ta ):

In [17]:
#| export
def gen_all_stats( 
    ta:object   # Object of class `TEST_AREA`.
):
    """
    Generate stats for all of the datasets within the `ta` class.
    """
    print(f"\nThe test area is {ta.area:2.1f} square meters. ({ta.area_width} by {ta.area_length} meters)\n")
    gen_stats( header=True)
    for i in ta.data_list:
        #gen_stats(values=i.df['ELEVATION'], name=i.name, area=hx4_a.area)
        gen_stats(data_obj=i,  area=ta.area)

## Test Files

### Bulldog & CZMIL

In [18]:
if 'google.colab' in str(get_ipython()):
    data_path = '/content/drive/MyDrive/Projects/2024-0209-NOAA-RSD-Bulldog/'
else:
    data_path = "../work/data/buldog/"
a10bdd    = data_path + "A-10m-bulldog-deep.txt"
a10bds    = data_path + "A-10m-bulldog-shallow.txt"
b10dd     = data_path + "B-10m-bulldog-deep.txt"
b10ds     = data_path + "B-10m-bulldog-shallow.txt"
c22bdd    = data_path + "C-22m-bulldog-deep.txt"
d30bdd    = data_path + "D-30m-bulldog-deep.txt"
a10cz     = data_path + "A-10m-czmil.txt"
b10cz     = data_path + "B-10m-czmil.txt"
c22cz     = data_path + "C-22m-czmil.txt"
d30cz     = data_path + "D-30m-czmil.txt"

In [31]:
bd_a = TEST_AREA()
bd_a.area_name      = "Area B"
bd_a.title          = f"{bd_a.area_name} ~10 meter depth. 1cm bins."
bd_a.area_width     = 3
bd_a.area_length    = 70
bd_a.area           = bd_a.area_width * bd_a.area_length

bd_a.bin_size       = 0.02
bd_a.density        = True
if bd_a.colab:
    bd_a.data_path  = "/content/drive/MyDrive/Projects/2024-0209-NOAA-RSD-Bulldog/"
else:
    bd_a.data_path      = "../data/bulldog/"
    
width = 3
#data( bd_a, offset=26.0, fn = bd_a.data_path + "A-30m-sonar.txt",          color = "red",   width = width, name = "SONAR", ref=True )
data( bd_a, offset=26.0, fn = bd_a.data_path + "B-10m-czmil.txt",           color = "green", width = width, name = "CZMIL", ref=True  )
data( bd_a, offset=26.0, fn = bd_a.data_path + "B-10m-bulldog-deep.txt",    color = "blue",  width = width, name = "Bulldog-D",       )
data( bd_a, offset=26.0, fn = bd_a.data_path + "B-10m-bulldog-shallow.txt", color = "Red",   width = width, name = "Bulldog-S",       )
gen_all_stats( bd_a )
plot_hists( bd_a )


The test area is 210.0 square meters. (3 by 70 meters)

                        Ref            Std                                 Total
    Data Source  Ref    Dif  Mean(m)  Dev(m)   Min(m)    Max(m)   P2P(m)  Points  Points/m  Scale  Offset(m)
          CZMIL <--   0.000 16.231    0.053   16.076    16.434    0.358     878     4.181  1.0000 26.000
      Bulldog-D       0.037 16.193    0.089   15.991    16.482    0.491      58     0.276  1.0000 26.000
      Bulldog-S       0.046 16.184    0.073   16.022    16.296    0.274      35     0.167  1.0000 26.000


Loading BokehJS ...

### Hawkeye, CZMIL, Sonar

In [21]:
hx4_a = TEST_AREA()
hx4_a.area_name      = "Area A"
hx4_a.title          = f"{hx4_a.area_name} ~30 meter depth. 1cm bins."
hx4_a.area_width     = 2
hx4_a.area_length    = 45
hx4_a.area           = hx4_a.area_width * hx4_a.area_length

hx4_a.bin_size       = 0.01
hx4_a.data_path      = "../data/ch4x/"
width = 3
data( hx4_a, offset=26.0, fn = hx4_a.data_path + "A-30m-sonar.txt", color = "red",   width = width, name = "SONAR", ref=True       )
data( hx4_a, offset=26.0, fn = hx4_a.data_path + "A-30m-czmil.txt", color = "green", width = width, name = "CZMIL"                 )
data( hx4_a, offset=26.0, fn = hx4_a.data_path + "A-30m-hx4.txt",   color = "blue",  width = width, name = "Hawkeye 4x", scale=1.0175 )

gen_all_stats( hx4_a )
plot_hists( hx4_a )


The test area is 90.0 square meters. (2 by 45 meters)

                        Ref            Std                                 Total
    Data Source  Ref    Dif  Mean(m)  Dev(m)   Min(m)    Max(m)   P2P(m)  Points  Points/m  Scale  Offset(m)
          SONAR <--   0.000 -29.928    0.087  -30.161   -29.739    0.422     684     7.600  1.0000 26.000
          CZMIL       0.070 -29.998    0.206  -30.621   -29.470    1.151     168     1.867  1.0000 26.000
     Hawkeye 4x       0.016 -29.944    0.099  -30.220   -29.701    0.519     294     3.267  1.0175 26.000


Loading BokehJS ...